In [1]:

import operator
import copy
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


In [2]:
### target_files

cn_target_file='./very_new_cn_all'
cn_source_file='./new_cn_source'
vm_file='./very_new_vms'

In [3]:
### res_s + examples
cpu_per_cn = 56
ram_per_cn = 516990
#ram_per_cn = 493040
test_machine = 'site-liferay-10'
test_hypervisor = 'cn-d720.egron.tnx'
test_hypervisor_alter = 'cn-dc6c.egron.tnx'
cpu_coef = 0.0002
ram_coef = 1.5

In [4]:
with open(vm_file, 'r') as f:
    vml = f.readlines()
    vml = [i.replace('\n','').split(',') for i in vml]
    
f.close()
with open(cn_target_file, 'r') as f:
    hpl = f.readlines()
    hpl = [i.replace('\n','').split(',') for i in hpl]
    
f.close()

In [5]:
def vm_dict_create(sl):
    'создаем словарь виртуальных машин'
    
    rd = {}
    for i in sl:
        if i[1]=='':
            crit='E'
        else:
            crit=i[1]
        rd[i[0]] = {'name': i[0],\
                 'cpu' : int(i[2]),\
                 'ram' : int(i[3]),\
                 'cn_name':i[4],\
                 'crit_rate':crit


        }
    return rd, copy.deepcopy(rd)


def hyper_dict_create(c,r,hl,sl):
    'создаем  словарь гипервизоров'
    rd= {}
    for i in hl:
        h_vm_list = [s[0] for s in sl if s[4] == i[0]]
        rd[i[0]] = { 'name' : i[0],\
                         'cpu'  : c,\
                         'ram'  : r,\
                         'vms'  : h_vm_list,\
                         'vms_count' : len(h_vm_list)


            }
    return rd

def hyper_dict_resources_create(hd,vd):
    rd = copy.deepcopy(hd)
    for i in rd.keys():
        #print([vd[j]['ram'] for j in rd[i]['vms']] )
        sum_ram_perc = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))/rd[i]['ram'] 
        sum_cpu_perc = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))/rd[i]['cpu']
        rd[i]['perc_cpu'] = sum_cpu_perc
        rd[i]['perc_ram'] = sum_ram_perc
        rd[i]['free_cpu'] = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))
        rd[i]['free_ram'] = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))
    return rd, copy.deepcopy(rd)

In [6]:
def count_over (h,v):
    vm_vcpu_sum = sum([v[i]['cpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum],\
        [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 

In [7]:
vmd, vmd_in_use = vm_dict_create(sl=vml)
hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hpd_res,vmd)

In [8]:
#hpd.keys()

In [9]:
#vmd[test_machine],hpd[test_hypervisor],hpd_res[test_hypervisor],hpd_res_in_use[test_hypervisor]

In [10]:
l = [hpd_res[h]['perc_ram'] for h in hpd_res.keys()]
m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]



print("Всего машин:", len(vml))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


Всего машин: 1654
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 1.338950463258477
Наименее забитый : 0.3802936227006325
Средняя загруженность : 0.8849156198684491

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 4.107142857142858
Наименее забитый : 1.3571428571428572
Средняя загруженность : 2.8500939849624056


### Заполняем словарь hpd_in_use информацией о заполненности

In [11]:
#hpd_res_in_use[test_hypervisor_alter]['vms']

In [12]:
#vmd

### Распределение в случае разогретого словаря



In [42]:
def start_iter_distribution(iterations=10):
    rd={}
    iter_num = 0
    hd = copy.deepcopy(hpd_res_in_use)
    vd = copy.deepcopy(vmd_in_use)
    while iter_num < iterations:
        rd[iter_num] = {}
        #ram_list = [[i,hd[i]['perc_ram']]for i in hd.keys()]
        #worst_cn = sorted(ram_list, key = lambda x: float(x[1]))[:1][0][0]
        #### Посчитаем score для всех с учетом двух значений 
        scored_list = [[i,((ram_coef * hpd_res_in_use[i]['perc_ram'] + cpu_coef * hpd_res_in_use[i]['perc_cpu']))]\
                       for i in hpd_res_in_use.keys()]
        worst_cn = sorted(scored_list, key = lambda x: float(x[1]))[:1][0][0]
        #print(worst_cn)
        mean_memory =  mean([vmd[j]['ram'] for j in hd[worst_cn]['vms']])
        #cand_list = [[vmd[j]['ram'],vmd[j]['cpu'], j ]for j in hd[worst_cn]['vms'] if vmd[j]['ram'] <= mean_memory ]
        cand_list = [[vmd[j]['ram'],vmd[j]['cpu'], j ]for j in hd[worst_cn]['vms'] if vmd[j]['ram'] <= mean_memory ]
        cand = sorted(cand_list, key=operator.itemgetter(0, 1))[:1]
        #print(iter_num,worst_cn,cand[0],cand_list[:3])
        ##print(iter_num,worst_cn,sorted(scored_list, key = lambda x: float(x[1]))[:3])
       # print(iter_num,worst_cn,cand[0],cand_list[:2])
        vm_to_migrate = cand[0][2]
        vm, ex, new, bc = migrate_vm(v=vm_to_migrate,hd=hd,vd=vd)
        
        rd[iter_num]['from']=ex
        rd[iter_num]['to']=new
        rd[iter_num]['vm']=vm
        rd[iter_num]['cand_hyp_from']=sorted(scored_list, key = lambda x: float(x[1]))[:3]
        rd[iter_num]['cand_vm']=sorted(cand_list,reverse=False)[:2]
        rd[iter_num]['cand_hyp_to']= bc
        ##print(iter_num,worst_cn,new,cand[0],sorted(cand_list,reverse=False)[:2])
        iter_num = iter_num + 1
        #print(iter_num,worst_cn, cand)
        #print(iter_num,sorted(ram_list, key = lambda x: int(x[1]))[:3])
        #print(iter_num,vm_to_migrate,vmd[vm_to_migrate]['cn_name'],vmd_in_use[vm_to_migrate]['cn_name'])

    return rd
        
    
    
def migrate_vm(v,hd,vd):
    ex_ = vd[v]['cn_name']
    best_cand = find_best_hid_by_score(v,hd)
    cn_best_cand = best_cand[0][0]
    #print('\t',v,vmd[v]['cn_name'],vmd_in_use[v]['cn_name'],vd[v]['cn_name'])
    calculate_after_loc(exh=ex_,hd=hd,vd=vd,h=cn_best_cand,v=v)
    #print('\t',v,vmd[v]['cn_name'],vmd_in_use[v]['cn_name'],vd[v]['cn_name'])
    return v, ex_, cn_best_cand, best_cand

def find_best_hid_by_score(v,hd):
    rd={}
    for h in hd.keys(): 
        if h == vmd_in_use[v]['cn_name']:
            continue
        #free_ram = hd[h]['free_ram']
        #free_cpu = hd[h]['free_cpu']
        free_ram = hd[h]['perc_ram']
        free_cpu = hd[h]['perc_cpu']
        if free_ram < 0:
            continue
        score = count_score(free_cpu,free_ram)
        rd[h]={#'mply': score_mply,\
                              # 'score_az':score_az,\
                              # 'score_hyp':score_hyp, 
                               'score': score,\
                               'id' : h,\
                               'ram': hd[h]['free_ram'],\
                               'cpu': hd[h]['free_cpu'],\
                               'vm_count' : hd[h]['vms']
                              }
    ##### исходя из того, что словарь приходит разогретым - мы можем позволить себе выбрать просто первого кандидата
    scoring_list = [[i, rd[i]['score'], rd[i]['ram'], rd[i]['cpu']] for i in rd.keys() ]
    best_buddy = sorted(scoring_list, key = lambda x: float(x[1]), reverse=True)[:5]
    return best_buddy

        
def count_score(pc,pr):
    
    res = (cpu_coef * pc + ram_coef * pr)/2

    ### score = 1.2 * az * 2.5 * hz * (1 - mw)
    ### Учитываем ресурсы
    score = res
    #print(res,az,hz,mw,score)
    return score
    '''    
def calculate_after_loc(v,h,exh,hd,vd):
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ## отнимаем абсолютные и относительные значения ресурсов
    hd[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hd[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hd[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hd[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hd[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hd[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hd[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hd[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hd[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    hd[exh]['vms'].remove(v)
    vd[v]['cn_name'] = h
    
'''
def calculate_after_loc(v,h,exh,hd,vd):
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ### Изменения во временных словарях после локации
    ###  # заполняем значения для выбранного кандидата
    ## отнимаем абсолютные и относительные значения ресурсов
    hd[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hd[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hd[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hd[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hd[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hd[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hd[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hd[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hd[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hd[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    hd[exh]['vms'].remove(v)
    vd[v]['cn_name'] = h

    
    
    ## отнимаем абсолютные и относительные значения ресурсов
    hpd_res_in_use[h]['free_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) 
    hpd_res_in_use[h]['perc_cpu'] = (hd[h]['free_cpu'] - vd[v]['cpu']) / cpu_per_cn
    hpd_res_in_use[h]['free_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) 
    hpd_res_in_use[h]['perc_ram'] = (hd[h]['free_ram'] - vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hpd_res_in_use[h]['vms_count'] = (hd[h]['vms_count'] + 1)
    ## обновляем список машин на ВУ
    hpd_res_in_use[h]['vms'].append(v)
    ### # заполняем значения для ex-ВУ
    ## прибавляем ресурсы 
    hpd_res_in_use[exh]['free_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) 
    hpd_res_in_use[exh]['perc_cpu'] = (hd[exh]['free_cpu'] + vd[v]['cpu']) / cpu_per_cn
    hpd_res_in_use[exh]['free_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) 
    hpd_res_in_use[exh]['perc_ram'] = (hd[exh]['free_ram'] + vd[v]['ram']) / ram_per_cn
    
    ## обновляем счетчик ВМ на ВУ
    hpd_res_in_use[exh]['vms_count'] = (hd[exh]['vms_count'] - 1)
    ## обновляем список машин на ВУ
    #print(v,hpd_res_in_use[exh]['vms'])
    hpd_res_in_use[exh]['vms'].remove(v)
    vmd_in_use[v]['cn_name'] = h
    
    
    #gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] = gdictinuse[vmd[v]['groupid']]['az'][hpd[h]['az']] + 1 
    #gdictinuse[vmd[v]['groupid']]['hyp'][h] = gdictinuse[vmd[v]['groupid']]['hyp'][h] + 1
    #hdictinuse[h]['vms_ids'].append(v)
    #if vmd[v]['crit_rate'] == 2:
    #    hdictinuse[h]['vm_critical'] = hdictinuse[h]['vm_critical'] + 1
    #if vmd[v]['crit_rate'] == 1:
    #    hdictinuse[h]['vm_warning'] = hdictinuse[h]['vm_warning'] + 1

In [43]:
#hpd_res_in_use[test_hypervisor]

In [44]:
#find_best_hid_by_score('pg-attribute-standby-07',hpd_res_in_use)

In [45]:
#find_best_hid_by_score('pg-attribute-standby-07',hpd_res_in_use)

In [46]:
vmd, vmd_in_use = vm_dict_create(sl=vml)
hpd= hyper_dict_create(cpu_per_cn,ram_per_cn,hpl,vml)
hpd_res, hpd_res_in_use = hyper_dict_resources_create(hpd,vmd)

In [47]:
ram_list = [[i,hpd_res_in_use[i]['perc_ram']] for i in hpd_res_in_use.keys()]
cpu_list = [[i,hpd_res_in_use[i]['perc_cpu']] for i in hpd_res_in_use.keys()]
scored_list = [[i,((5 * hpd_res_in_use[i]['perc_ram'] + hpd_res_in_use[i]['perc_cpu'])/2)] for i in hpd_res_in_use.keys()]
worst_cn = sorted(ram_list, key = lambda x: float(x[1]))[:1][0][0]
sorted(ram_list, key = lambda x: float(x[1]))[:5],\
sorted(cpu_list, key = lambda x: float(x[1]))[:5],\
sorted(scored_list, key = lambda x: float(x[1]))[:5]

([['cn-e624.egron.tnx', -0.33895046325847694],
  ['cn-e13c.egron.tnx', -0.06165303003926575],
  ['cn-dce2.egron.tnx', -0.014116327201686686],
  ['cn-e8ae.egron.tnx', 0.001729240410839668],
  ['cn-e612.egron.tnx', 0.0254975918296292]],
 [['cn-e810.egron.tnx', -3.107142857142857],
  ['cn-e80c.egron.tnx', -2.9642857142857144],
  ['cn-e13c.egron.tnx', -2.7857142857142856],
  ['cn-e8b6.egron.tnx', -2.7142857142857144],
  ['cn-d720.egron.tnx', -2.642857142857143]],
 [['cn-e13c.egron.tnx', -1.5469897179553072],
  ['cn-e624.egron.tnx', -1.4723761581461923],
  ['cn-e80c.egron.tnx', -1.4183988775687841],
  ['cn-e810.egron.tnx', -1.3907926514190658],
  ['cn-e8b6.egron.tnx', -1.2735919180531263]])

In [48]:


l = start_iter_distribution(iterations=1000)
ll = copy.deepcopy(l)

In [62]:
[[i,ll[i]['vm']] for i in ll.keys()]

[[0, 'sf-bal-01'],
 [1, 'ppoz-kuvd-migration-service-09'],
 [2, 'pg-attribute-79'],
 [3, 'pg-attribute-standby-12'],
 [4, 'pg-attribute-standby-79'],
 [5, 'migr-arc-85'],
 [6, 'migr-node-005'],
 [7, 'cia-sign-bal-02'],
 [8, 'lk-rd-02'],
 [9, 'cia-core-01'],
 [10, 'lk-backend-03'],
 [11, 'isurz-xml-storage-39'],
 [12, 'isurz-api-05'],
 [13, 'ppoz-bus-ext-01'],
 [14, 'cia-idp-core-09'],
 [15, 'ppoz-gmp-process-02'],
 [16, 'isurz-xml-storage-89'],
 [17, 'ppoz-service-bal-01'],
 [18, 'cia-sign-bal-01'],
 [19, 'ppoz-gmp-bal-01'],
 [20, 'pg-mq-01'],
 [21, 'cia-core-bal-02'],
 [22, 'isurz-xml-storage-21'],
 [23, 'cia-core-bal-01'],
 [24, 'ppoz-service-notify-02'],
 [25, 'migr-gkn-oracle-01'],
 [26, 'cia-sign-10'],
 [27, 'cia-sign-05'],
 [28, 'ir-app-gi-bal-01'],
 [29, 'cite-bal-outer-01'],
 [30, 'ppoz-nsi-adapter-01'],
 [31, 'isurz-xml-storage-60'],
 [32, 'ppoz-kuvd-bal-02'],
 [33, 'pmuf-arm-sc-balancer-01'],
 [34, 'lk-rd-03'],
 [35, 'site-liferay-srv-07'],
 [36, 'mgmt-repository-01'],
 [37, 

In [49]:
#[[i,ll[i]['vm']] for i in ll.keys()]
s = [ll[i]['vm'] for i in ll.keys()if i < 131]

len(set(s))

122

In [50]:
ll[1],ll[2],ll[3],ll[4]

({'cand_hyp_from': [['cn-e624.egron.tnx', -0.48489305775464014],
   ['cn-e13c.egron.tnx', -0.09303668791604147],
   ['cn-dce2.egron.tnx', -0.021660205088244316]],
  'cand_hyp_to': [['cn-ed8e.egron.tnx', 0.4528527501222738, 316286, -22],
   ['cn-dc8e.egron.tnx', 0.2507503344911341, 172926, -64],
   ['cn-daa0.egron.tnx', 0.23883044449602506, 164734, -84],
   ['cn-dc7c.egron.tnx', 0.22100775236050432, 152446, -82],
   ['cn-e4c2.egron.tnx', 0.209152148079681, 144254, -66]],
  'cand_vm': [[16384, 4, 'ppoz-kuvd-migration-service-09'],
   [16384, 8, 'pg-attribute-79']],
  'from': 'cn-e624.egron.tnx',
  'to': 'cn-ed8e.egron.tnx',
  'vm': 'ppoz-kuvd-migration-service-09'},
 {'cand_hyp_from': [['cn-e624.egron.tnx', -0.4016823992174483],
   ['cn-e13c.egron.tnx', -0.09303668791604147],
   ['cn-dce2.egron.tnx', -0.021660205088244316]],
  'cand_hyp_to': [['cn-ed8e.egron.tnx', 0.41124742085367777, 299902, -26],
   ['cn-dc8e.egron.tnx', 0.2507503344911341, 172926, -64],
   ['cn-daa0.egron.tnx', 0.2388

In [51]:
hpd_res[test_hypervisor],vmd['pg-attribute-standby-07']

({'cpu': 56,
  'free_cpu': -148,
  'free_ram': 17278,
  'name': 'cn-d720.egron.tnx',
  'perc_cpu': -2.642857142857143,
  'perc_ram': 0.033420375635892376,
  'ram': 516990,
  'vms': ['isurz-xml-storage-21',
   'pg-attribute-73',
   'pg-attribute-standby-07',
   'pg-attribute-standby-25',
   'cia-sign-06',
   'cia-sign-08',
   'isurz-elasticsearch-07',
   'migr-arc-5',
   'migr-arc-xx',
   'migr-jst-oracle-15',
   'migr-node-022',
   'pkurp-worker-020',
   'pmo-db-fed-01',
   'ppoz-bus-command-04',
   'ppoz-bus-command-08',
   'ppoz-bus-data-08',
   'ppoz-bus-data-10',
   'ppoz-db-proc-gmp-03',
   'ppoz-db-proc-gmp-05',
   'ppoz-db-proc-pkurp-05',
   'ppoz-db-proc-popd-05',
   'ppoz-db-proc-ppoz-05',
   'ppoz-db-request-07',
   'ppoz-popd-process-04',
   'ppoz-process-core-03',
   'ppoz-process-core-05',
   'ppoz-service-external-03',
   'pg-attribute-standby-89'],
  'vms_count': 28},
 {'cn_name': 'cn-d720.egron.tnx',
  'cpu': 8,
  'crit_rate': '1',
  'name': 'pg-attribute-standby-07',
 

In [52]:
#migrate_vm(hd=hpd_res_in_use,vd=vmd_in_use,v='pg-attribute-standby-07')

In [53]:
hpd_res_in_use[test_hypervisor],vmd_in_use['pg-attribute-standby-07']

({'cpu': 56,
  'free_cpu': -128,
  'free_ram': 58238,
  'name': 'cn-d720.egron.tnx',
  'perc_cpu': -2.2857142857142856,
  'perc_ram': 0.11264821369852415,
  'ram': 516990,
  'vms': ['pg-attribute-73',
   'pg-attribute-standby-07',
   'pg-attribute-standby-25',
   'isurz-elasticsearch-07',
   'migr-arc-5',
   'migr-arc-xx',
   'migr-jst-oracle-15',
   'migr-node-022',
   'pmo-db-fed-01',
   'ppoz-bus-command-04',
   'ppoz-bus-command-08',
   'ppoz-bus-data-08',
   'ppoz-bus-data-10',
   'ppoz-db-proc-gmp-03',
   'ppoz-db-proc-gmp-05',
   'ppoz-db-proc-pkurp-05',
   'ppoz-db-proc-popd-05',
   'ppoz-db-proc-ppoz-05',
   'ppoz-db-request-07',
   'ppoz-popd-process-04',
   'ppoz-process-core-03',
   'ppoz-process-core-05',
   'ppoz-service-external-03',
   'pg-attribute-standby-89'],
  'vms_count': 23},
 {'cn_name': 'cn-d720.egron.tnx',
  'cpu': 8,
  'crit_rate': '1',
  'name': 'pg-attribute-standby-07',
  'ram': 16384})

In [54]:
#scoring_list = [[i, score_for_test[i]['score'], score_for_test[i]['ram']] for i in score_for_test.keys() ]

In [55]:
#sorted(scorint_list, key = lambda x: float(x[1]), reverse=True)[:1]

In [56]:
#ram_test = [[i,hpd_res_in_use[i]['free_ram']]for i in hpd_res_in_use.keys()]


In [57]:
#cn_test = sorted(ram_test, key = lambda x: int(x[1]))
#w_cn = cn_test[:1][0][0]
#hpd_res_in_use[w_cn]['vms']
#mean_memory =  mean([vmd[j]['ram'] for j in hpd_res_in_use[w_cn]['vms']])
#cand_list = [[vmd[j]['ram'], j ]for j in hpd_res_in_use[w_cn]['vms'] if vmd[j]['ram'] > 0]
#cand = sorted(cand_list)[:1]
#cand[0][1]

In [58]:
#cn_test
#mean([vmd[j]['ram'] for j in hpd_res_in_use[cn_test]['vms']])

In [59]:
l = [hpd_res[h]['perc_ram'] for h in hpd_res.keys()]
m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]



print("Всего машин:", len(vml))
print("Всего машин:", len(vmd.keys()))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


Всего машин: 1654
Всего машин: 1652
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 1.338950463258477
Наименее забитый : 0.3802936227006325
Средняя загруженность : 0.8849156198684491

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 4.107142857142858
Наименее забитый : 1.3571428571428572
Средняя загруженность : 2.8500939849624056


In [60]:
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hpd_res_in_use,vmd_in_use)

In [61]:
l = [hpd_res_in_use[h]['perc_ram'] for h in hpd_res_in_use.keys()]
m = [hpd_res_in_use[h]['perc_cpu'] for h in hpd_res_in_use.keys()]



print("Всего машин:", len(vmd_in_use.keys()))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
print("Всего ВУ:", len(hpl))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


Всего машин: 1652
Необходимо машинам vCPU : 12118 , RAM : 34728472 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 2.8472744360902253 , RAM : 0.8838731483149934
Забитость ВУ по RAM
Самый забитый : 0.9190429215265286
Наименее забитый : 0.8715062186889495
Средняя загруженность : 0.8834561596936112

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 3.9285714285714284
Наименее забитый : 0.5714285714285714
Средняя загруженность : 2.840225563909775
